# Comparaison between Python and Matlab for denoising

## Google Colab Integration

In [12]:
# !git clone https://github.com/Stage-SuperResolution/NAME-REPO.git
# !mv NAME-REPO/* .
# !rm -rf NAME-REPO
# !pip install git+https://github.com/akhaten/lasp.git@COMPLETE-COMMIT

## Import

In [13]:
# Lasp module

import sys
sys.path.append('../../..')

import lasp.io
import lasp.filters.linear
import lasp.noise
import lasp.convert
import lasp.algorithm.experimental
import lasp.thresholding
import lasp.differential
import lasp.utils

# Other

import scipy.signal
import scipy.io.matlab
import numpy
import pandas
import tqdm
import pathlib
import matplotlib.pyplot

IMAGE_PATH = pathlib.Path('../0-Images')
PYMAT_PATH = pathlib.Path('./1-PythonVsMatlab')
if not(PYMAT_PATH.exists()):
    PYMAT_PATH.mkdir()

## Test On

In [14]:
def mumford_shah_denoise_v1(
    y: numpy.ndarray,
    alpha: float,
    beta: float,
    sigma: float,
    nb_iterations: int,
    tolerance: float,
    error_history: list[float] = None
) -> numpy.ndarray:

    """Mumford Shah
    
    Solve argmin_{x} { (alpha/2) || y - x ||^2 + (beta/2) || nabla y ||^2 + || nabla y ||_1 }
    """

    Dx = lasp.differential.dx
    Dy = lasp.differential.dy
    Dxt = lasp.differential.dxT
    Dyt = lasp.differential.dyT

    # Build kernel
    laplacian = lasp.filters.linear.laplacian()
    lap_diag = lasp.utils.fourier_diagonalization(
        kernel = laplacian,
        shape_out = y.shape 
    )
   

    uker = alpha + (beta+sigma) * lap_diag

    rhs1fft = alpha * numpy.fft.fft2(y)

    # Initialization
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(y)
    d_y=numpy.zeros_like(y)
    b_x=numpy.zeros_like(y)
    b_y=numpy.zeros_like(y)

    for _ in range(0, nb_iterations):

        rhs2 = sigma*Dxt(d_x-b_x)+sigma*Dyt(d_y-b_y)
        rhsfft = rhs1fft + numpy.fft.fft2(rhs2)

        u0=numpy.copy(u)
        
        u_fft = rhsfft / uker
        u = numpy.real(numpy.fft.ifft2(u_fft))    

        err = numpy.linalg.norm(u-u0, 'fro') / numpy.linalg.norm(u, 'fro')
        
        if not(error_history is None):
            error_history.append(err)

        if err < tolerance:
            break
        
        u_dx, u_dy = Dx(u), Dy(u)

        d_x, d_y = lasp.thresholding.multidimensional_soft(
            d = numpy.array([ u_dx + b_x, u_dy + b_y ]),
            epsilon = 1/sigma
        )

        b_x += (u_dx - d_x)
        b_y += (u_dy - d_y)

    u_normalized = lasp.utils.normalize(u)

    return u_normalized

## Dataset utils

In [15]:
def image_from(params: pandas.Series) -> numpy.ndarray:

    image_path = params['image']
    noise = params['noise']
    
    img = lasp.io.read(image_path)
    out = numpy.copy(img) # x

    if pandas.notna(noise):
        out = lasp.noise.awgn(out, snr=noise) # x + n

    return out # y = x + n

def add(
    dataset: pandas.DataFrame,
    image: pathlib.Path,
    alpha: float,
    beta: float,
    sigma: float,
    tol: float,
    iterations: int,
    noise: float
) -> pandas.DataFrame:

    to_add = pandas.DataFrame(
        {
            'image' : [image], 
            'alpha' : [alpha],
            'beta' : [beta],
            'sigma' : [sigma],
            'tol' : [tol], 
            'iterations' : [iterations], 
            'noise' : [noise]
        }
    )

    return pandas.concat([dataset, to_add], ignore_index=True)

def process_from_index(dataset: pandas.DataFrame, index: int) -> tuple[numpy.ndarray, numpy.ndarray]:
   

    params = dataset.loc[index] # Get params from dataset


    out = image_from(params)
    normalized = lasp.utils.normalize(out) 

    # Hyper-parameters
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    # Iterative parameters
    tol = params['tol']
    iterations = params['iterations']

    errors = []

    res = mumford_shah_denoise_v1(
        normalized, 
        alpha, beta, sigma,
        iterations, tol, 
        errors
    )

    return res, errors

In [16]:
def make_dataset() -> pandas.DataFrame:    
    

    dataset = pandas.DataFrame(
        columns = [
            'image' , # Image path
            'alpha', 'beta', 'sigma', # Hyper-parameters
            'tol', 'iterations' # Iterative parameters
        ]
    )

    dataset = add(
        dataset = dataset, 
        image = IMAGE_PATH / 'Baboon.bmp', 
        alpha = 100, beta = 1, sigma = 2, 
        tol = 1e-4, iterations = 300, noise = lasp.convert.snrdb_to_snr(30)
    )

    return dataset

In [17]:
DATASET_PATH = PYMAT_PATH / pathlib.Path('dataset.pkl')
dataset = make_dataset()
dataset

,image,alpha,beta,sigma,tol,iterations,noise
0,../0-Images/Baboon.bmp,100,1,2,0.0001,300,31.622777


In [18]:
pandas.to_pickle(dataset, DATASET_PATH)

In [19]:
MAT_PATH = PYMAT_PATH / 'datas.mat'

def make_mat(dataset: pandas.DataFrame, index: int) -> dict:
    datas = dataset.loc[index].to_dict()
    img = image_from(dataset.loc[index])
    normalized = lasp.utils.normalize(img)
    datas['image'] = normalized
    return datas

scipy.io.matlab.savemat(MAT_PATH, make_mat(dataset, index=0))

## Process Dataset

In [20]:
begin = 0
dataset = pandas.read_pickle(DATASET_PATH)
dataset_filtered = dataset.loc[begin:]

RESULTS_PATH = PYMAT_PATH / pathlib.Path('./results')
if not(RESULTS_PATH.exists()):
    RESULTS_PATH.mkdir()

for i in tqdm.tqdm(dataset_filtered.index):


    RES_CURRENT_PATH = RESULTS_PATH / str(i)
    if not(RES_CURRENT_PATH.exists()):
        RES_CURRENT_PATH.mkdir()

    res, errors = process_from_index(dataset_filtered, i)

    lasp.io.save(res, RES_CURRENT_PATH / 'result.npy')
    lasp.io.save(res, RES_CURRENT_PATH / 'result.png')
    lasp.io.save(errors, RES_CURRENT_PATH / 'errors.npy')

100%|██████████| 1/1 [00:08<00:00,  8.85s/it]


## Comparaison with Matlab results

In [21]:
# res_mat = scipy.io.matlab.loadmat(pathlib.Path('./res.mat'))[???]
# res_py = lasp.io.read(RESULTS_PATH / '0' / 'result.npy')

In [22]:
# print(numpy.max(numpy.abs(res_py-res_mat)))